In [32]:
import numpy as np
import tensorflow as tf
import pandas as pd
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [33]:
print("Number of GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Number of GPUs Available:  0


In [34]:
tweets_df = pd.read_csv('data/Tweets.csv')
tweets_df.head(10)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)
6,570300616901320704,positive,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
7,570300248553349120,neutral,0.6340,NaN,NaN,Virgin America,NaN,pilot,NaN,0,@VirginAmerica Really missed a prime opportuni...,NaN,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
8,570299953286942721,positive,0.6559,NaN,NaN,Virgin America,NaN,dhepburn,NaN,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",NaN,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,570295459631263746,positive,1.0000,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,"@VirginAmerica it was amazing, and arrived an ...",NaN,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


In [35]:
tweets_df[['airline_sentiment_confidence']].describe()

,airline_sentiment_confidence
count,14640.000000
mean,0.900169
std,0.162830
min,0.335000
25%,0.692300
50%,1.000000
75%,1.000000
max,1.000000


In [36]:
def sentiment_classifier(rating):
    if rating >= 5:
        return int(2)
    elif rating > 3:
        return int(1)
    if rating <= 3:
        return int(0)

In [37]:
tweets_df['sentiment'] = tweets_df['airline_sentiment_confidence'].apply(sentiment_classifier)

In [38]:
def fix_floats(score):
    return int(score)

In [39]:
tweets_df['text'] = tweets_df['airline_sentiment_confidence'].dropna()
tweets_df['text'].sample(10)

11329    1.0000
10771    1.0000
994      0.6534
6162     1.0000
2883     1.0000
1158     0.6448
13950    1.0000
726      1.0000
46       0.6799
8646     1.0000
Name: text, dtype: float64

In [40]:
a = tweets_df['airline_sentiment_confidence'].dropna()
b = a.apply(fix_floats)
b

0        1
1        0
2        0
3        1
4        1
        ..
14635    0
14636    1
14637    1
14638    1
14639    0
Name: airline_sentiment_confidence, Length: 14640, dtype: int64

In [41]:
tweets_df['text'] = b.apply(fix_floats)

In [48]:
tweets_df

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,sentiment
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,1,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),0
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,0,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),0
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,0,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),0
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,1,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),0
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,1,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,0,NaN,2015-02-22 12:01:01 -0800,NaN,NaN,0
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,1,NaN,2015-02-22 11:59:46 -0800,Texas,NaN,0
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,1,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN,0
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,1,NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada),0


In [59]:
test_text = tweets_df['airline_sentiment_confidence']
test_review = tweets_df['sentiment']

In [60]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(test_text)

AttributeError: 'float' object has no attribute 'lower'

In [54]:
test_text = tokenizer.texts_to_sequences(test_text)

AttributeError: 'float' object has no attribute 'lower'

In [55]:
test_text = tokenizer.sequences_to_matrix(test_text)
test_text

ValueError: Specify a dimension (`num_words` argument), or fit on some text data first.

In [47]:
model = Sequential()

In [62]:
model.add(Dense(677, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(557, activation='relu'))
model.add(Dropout(0.2))
# model.add(Dense(419, 'linear'))
# model.add(Dropout(0.3))
model.add(Dense(281, activation='elu'))
model.add(Dropout(0.4))
model.add(Dense(167, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(61, activation='softsign'))
model.add(Dense(3, activation='softmax'))

In [63]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [65]:
test_sentiment = keras.utils.to_categorical(test_sentiment, 3)

NameError: name 'keras' is not defined